# Techniques for Handling Big Data in Python
### by [Jason DeBacker](https://jasondebacker.com), October 2024

This notebook offers some techniques for handling big data in Python.  The techniques include:
* Chunking data
* Reading and testing code with a sample of data
* Using efficient data types
* `pyreadstat` for reading SAS/Stata/SPSS files: [https://github.com/Roche/pyreadstat](https://github.com/Roche/pyreadstat)
* SQL in Python

In [46]:
import pandas as pd
import pyreadstat
import os
# from memory_profiler import profile
import memory_profiler

In [47]:
# to allow use of memit for memory profiling
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [48]:
# set paths to data
CUR_DIR = os.getcwd()
DATA_DIR = os.path.join(CUR_DIR, 'data')
DATA_FILE = os.path.join(DATA_DIR, 'trade_data_1962_to_2022.csv')

## Making Pandas More Efficient

#### 1. Test code on subset of data

Use the `nrows` argument in the Pandas `read_XXX` command (note, not all read commands have this option) to read in only a subset of the data.  This is useful for testing code on a smaller subset of the data before running on the full dataset.

In [49]:
df = pd.read_csv(DATA_FILE, nrows=10_000)
df.shape

(10000, 9)

### 2. Make more efficient data types

In [50]:
# to see datatypes for each column
df.dtypes

country_id              int64
partner_country_id      int64
year                    int64
product_id              int64
export_value          float64
import_value          float64
coi                   float64
eci                   float64
pci                   float64
dtype: object

In [51]:
df.memory_usage(deep=True)  # memory usage in bytes

Index                   132
country_id            80000
partner_country_id    80000
year                  80000
product_id            80000
export_value          80000
import_value          80000
coi                   80000
eci                   80000
pci                   80000
dtype: int64

In [53]:
df2 = df.copy()
df2[["country_id", "partner_country_id", "product_id", "year"]] = (
    df2[["country_id", "partner_country_id", "product_id", "year"]].apply(pd.to_numeric, downcast="unsigned")
)
df2[["export_value", "import_value", "coi", "eci", "pci"]] = (
    df2[["export_value", "import_value", "coi", "eci", "pci"]].apply(pd.to_numeric, downcast="float")
)

df2.dtypes

country_id             uint16
partner_country_id     uint16
year                   uint16
product_id             uint16
export_value          float32
import_value          float32
coi                   float32
eci                   float32
pci                   float32
dtype: object

In [54]:
df2.memory_usage(deep=True)

Index                   132
country_id            20000
partner_country_id    20000
year                  20000
product_id            20000
export_value          40000
import_value          40000
coi                   40000
eci                   40000
pci                   40000
dtype: int64

In [55]:
reduction = df2.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()
print(f"{reduction:0.2f}")

0.39


By changing a few data types, we reduce the memory usage by 61%.  This can be a big deal when working with big data.  And with strings, you can use the `category` data type in Pandas to reduce memory usage, sometimes very significantly.

However, be careful with making sure you don't loose important precision of extreme values by downcasting the data types.  We can check by doing a `df.describe()` before and after downcasting.

In [56]:
df2.describe() - df.describe()

,country_id,partner_country_id,year,product_id,export_value,import_value,coi,eci,pci
count,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
mean,0.0,0.0,0.0,0.0,-0.006275,0.008750,-7.187206e-09,5.327284e-08,3.308717e-08
std,0.0,0.0,0.0,0.0,-0.000410,-0.024651,-1.520906e-08,-4.549485e-08,-2.185737e-08
min,0.0,0.0,0.0,0.0,0.000000,0.000000,-4.472961e-08,2.032776e-08,9.503174e-09
25%,0.0,0.0,0.0,0.0,0.000000,0.000000,-4.795456e-09,2.289963e-10,2.861786e-08
50%,0.0,0.0,0.0,0.0,0.000000,0.000000,3.164053e-11,2.134323e-08,2.834473e-08
75%,0.0,0.0,0.0,0.0,0.000000,0.000000,7.077027e-09,-3.067474e-08,1.314545e-09
max,0.0,0.0,0.0,0.0,0.000000,0.000000,9.889221e-08,-3.009033e-08,1.314545e-09


### 3. Chunking data

Pandas has some built-in options to read data in "chunks", so you don't have to store it all in memory at once

In [9]:
def full_data():
    df = pd.read_csv(DATA_FILE)
    mean_exports = df["export_value"].mean()
    print(f"Mean exports: {mean_exports:,.2f}")

def chunked_data():
    chunksize = 100_000
    total = 0
    count = 0
    df = pd.read_csv(DATA_FILE, chunksize=chunksize)  # NOTE: using the chunksize argument returns an iterable list of dataframes
    for chunk in pd.read_csv(DATA_FILE, chunksize=chunksize):
        total += chunk["export_value"].sum()
        count += chunk["export_value"].count()
    print(f"Mean exports: {total/count:,.2f}")

In [10]:
%memit full_data()

Mean exports: 2,589,223.40
peak memory: 25005.86 MiB, increment: 23400.36 MiB


In [11]:
%memit chunked_data()

Mean exports: 2,589,223.40
peak memory: 2982.42 MiB, increment: 49.64 MiB


## Efficiently reading SAS, STATA, SPSS files

The `pyreadstat` package is a Python package that allows you to read SAS, Stata, and SPSS files into Pandas DataFrames very efficiently.  It is a wrapper around the `Readstat` C library.

In [14]:
df, meta = pyreadstat.read_sas7bdat(os.path.join(DATA_DIR, "child_u18.sas7bdat"))
df.head()

,HHID,year,cu18,kids
0,000001,1992.0,0.0,4.0
1,000001,1994.0,0.0,4.0
2,000002,1992.0,0.0,8.0
3,000002,1994.0,0.0,8.0
4,000002,1996.0,0.0,8.0


In [18]:
# Horse race between pandas and pyreadstat, reading a largish Stata file
%timeit df, meta = pyreadstat.read_dta(os.path.join(DATA_DIR, "sitc_country_country_product_year_4_2022.dta"))
%timeit df = pd.read_stata(os.path.join(DATA_DIR, "sitc_country_country_product_year_4_2022.dta"))

6.66 s ± 840 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
282 ms ± 42.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit df, meta = pyreadstat.read_sas7bdat(os.path.join(DATA_DIR, "child_u18.sas7bdat"))
%timeit df = pd.read_sas(os.path.join(DATA_DIR, "child_u18.sas7bdat"))

152 ms ± 27.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
40.1 ms ± 3.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Other advantages of pyreadstat
1. Read only a subset of columns (can't do this on SAS or STATA files in pd.read_sas or pd.read_stata)
2. Built in mulitprocessing
3. Can read metadata only

In [21]:


# 1. Read only a subset of columns
df, meta = pyreadstat.read_sas7bdat(os.path.join(DATA_DIR, "child_u18.sas7bdat"), usecols=["HHID", "year"])
df.head()

,HHID,year
0,000001,1992.0
1,000001,1994.0
2,000002,1992.0
3,000002,1994.0
4,000002,1996.0


In [29]:
# 2. Built in multiprocessing
%timeit df, meta = pyreadstat.read_dta(os.path.join(DATA_DIR, "sitc_country_country_product_year_4_2022.dta"))
%timeit df, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_dta, os.path.join(DATA_DIR, "sitc_country_country_product_year_4_2022.dta"), num_processes=4)

6.02 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.15 s ± 343 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
# Read metadata only
_, meta = pyreadstat.read_sas7bdat(os.path.join(DATA_DIR, "child_u18.sas7bdat"), metadataonly=True)  # note, still returns DF
print(meta.column_names)

['HHID', 'year', 'cu18', 'kids']


## SQL in Python

In [ ]:
# (NONWORKING IN THIS NOTEBOOK) Example of SQL in Python
# Connect to WRDS (Wharton Research Data Services) using the wrds package
import wrds
import pandas as pd
import os
db = wrds.Connection(wrds_username='jdebacker')


# Retrieve 2015 - 2017 Earnings Conference Call Transcripts with Full-text components
# loop over the companyid list
for year in range(2015, 2018):
    df_list = []
    # Loop over the company ids for a given year
    for companyid in companyid_list:
        sql_query = (
            "SELECT a.companyid, a.headline, a.companyname, a.mostimportantdateutc, "
            "a.mostimportanttimeutc, b.componenttext, b.transcriptComponentId, "
            "b.componentOrder, b.transcriptPersonId, b.transcriptId, "
            "c.companyName, c.proId, c.speakerTypeId, d.speakerTypeName, "
            "e.transcriptComponentTypeName "
            "FROM ciq_transcripts.wrds_transcript_detail AS a "
            "JOIN ciq_transcripts.ciqtranscriptcomponent as b ON a.transcriptid=b.transcriptid "
            "JOIN ciq_transcripts.ciqtranscriptperson as c ON b.transcriptPersonId=c.transcriptPersonID "
            "JOIN ciq_transcripts.ciqtranscriptspeakertype as d ON c.speakerTypeId=d.speakerTypeId "
            "JOIN ciq_transcripts.ciqtranscriptcomponenttype as e ON  b.transcriptComponentTypeId=e.transcriptComponentTypeId "
            "WHERE a.companyid={0} AND "
            "date_part('year',mostimportantdateutc)={1}; "
        ).format(companyid, year)

        print(
            "Running query for year {y} and companyid {c}".format(
                y=year, c=companyid
            )
        )
        data = db.raw_sql(sql_query)
        # drop duplicate observations of text components
        data.drop_duplicates(
            subset=[
                "componentorder",
                "proid",
                "headline",
                "mostimportanttimeutc",
                "mostimportantdateutc",
                "transcriptcomponentid",
                "componenttext",
            ],
            keep="first",
            inplace=True,
            ignore_index=False,
        )
        # append the data for a given company id to the list of dataframes
        df_list.append(data)